# Regularización

La regularización es un término que comprende diferentes métodos que fuerzan a los algoritmos de aprendizaje a crear un modelo  menos complejo. En la práctica, esto puede llevar a menudo a un sesgo ligeramente mayor pero reduce de forma significativa la varianza. Este término se conoce como el *bias-variance tradeoff*.

![bias-variance tradeoff](../../images/03_01_bias_variance.png)

[Fuente - From Understanding the Bias-Variance Tradeoff, por Scott Fortmann-Roe.](http://scott.fortmann-roe.com/docs/BiasVariance.html)

Queremos que nuestro modelo sea lo más simple posible pero no tan simple. Es por ello que deberemos de probar diferentes cosas para llegar a un óptimo:

![model complexity](../../images/03_02_model_complex.png)

A medida que añadimos más y más información a nuestro modelo estamos aumentando su complejidad. Este suele llevar a un incremento de la varianza mejorando nuestro sesgo pero esto conduce, también, a potencial sobreajuste. 

Por tanto, en la práctica debemos encontrar un óptimo donde no se nos dispare nada (sesgo, varianza, complejidad) y que nuestro modelo generalice bien. Pero no hay una forma de hacer esto que sea totalmente objetiva y analítica.

¿Cómo podemos lidiar con estas cosas?

Si nuestro modelo funciona mal podemos añadirle mayor información. A medida que añadimos más *features* a nuestro modelo su complejidad aumenta y debería ayudarnos a reducir el sesgo pero hay que hacerlo con cabeza para que no lleguemos a un sobreajuste.

Basicamente tenemos dos formas:
* Reducir la complejidad del modelo, es decir, quitar columnas de nuestro conjunto de datos.
* Usar **regularización**. 

En la regularización lo que hacemos es añadir un coeficiente de penalización que aumentará de valor a medida que el modelo sea más complejo y de esta forma reducirá la varianza del modelo.

Y ahora, veamos cómo funciona.

# Sobreajuste en la regresión lineal

Vamos a calcular la regresión de un modelo para ver qué efecto tiene el sobreajuste. Para ello, vamos a generar un nuevo dataset más complejo a partir de los datos de precios de viviendas de Boston:

In [1]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures

def load_extended_boston():
    boston = load_boston()
    X = boston.data

    X = MinMaxScaler().fit_transform(boston.data)
    X = PolynomialFeatures(degree=2).fit_transform(X)  # genera nuevas columnas para obtener un problema más complejo

    return X, boston.target

X, y = load_extended_boston()
print(X.shape, y.shape)

(506, 105) (506,)


Vemos que tenemos un dataset con 506 ejemplos (filas) y 105 *features* (columnas).

In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
linreg = LinearRegression().fit(X_train, y_train)

Y ahora calculamos qué bien está funcionando nuestra regresión lineal sobre los datos de entrenamiento:

In [3]:
print(linreg.score(X_train, y_train))

0.9520519609032732


¿Y con datos nunca antes vistos? 

In [4]:
print(linreg.score(X_test, y_test))

0.6074721959665728


¡Vaya! Este valor es mucho más bajo que en los datos de entrenamiento. Como hemos explicado, ahora podríamos mejorar el modelo mediante una regularización. Veamos cómo hacerlo.

# Regularización Ridge

Vamos a ver primero la regularización más utilizada habitualmente, **Ridge**:

$$ f(w, b) = \lambda||\textbf{w}||^2 +\frac{1}{N}\sum_{i=1}^{N}(y_i - (w*x_i+b))^2 $$

donde $ ||\textbf{w}|| = \sum_{j=1}^{D}(w_{j})^2 $, es la suma del valor al cuadrado de los pesos
y $ \lambda $ sigue siendo un hiperparámetro que controla la importancia de la regularización.

Si $ \lambda = 0 $ nos encontramos en el caso de la regresión lineal. En cambio, si $ \lambda >> 0 $ el algoritmo de aprendizaje intentará que todos los $ w_j $ estén muy cerca de 0 (pero no siendo cero).

En general, la regularización **Ridge** es conveniente cuando todas las columnas tienen importancia.

En sklearn podemos ejecutar esta regularización de la siguiente forma:

In [5]:
from sklearn.linear_model import Ridge
linreg_ridge = Ridge(alpha=1.0).fit(X_train, y_train)

Veamos qué error estamos teniendo en el dataset de entrenamiento:

In [6]:
print(linreg_ridge.score(X_train, y_train))

0.8857966585170942


Y ahora, con datos no utilizamos en el entrenamiento:

In [7]:
print(linreg_ridge.score(X_test, y_test))

0.7527683481744754


¿Qué ha sucedido?

Como podéis ver los valores son menor y mayor para los datos de entrenamiento y de prueba, respectivamente, cuando lo comparamos con el caso de la regresión lineal (0.952, 0.607). Recordad que la regresión lineal estaba sobreajustando.

Un modelo menos complejo es un modelo que funcionará un poco mejor en el entrenamiento pero generalizará mejor. Como estamos interesados en el rendimiento de la generalización deberíamos elegir el modelo *Ridge* en lugar del modelo de regresión lineal para este caso concreto.

Como expertos científicos de datos deberemos ajustar correctamente el parámetro $ \lambda $ de la regularización L2 que usa este modelo. $ \lambda $ es el parámetro `alpha` en `scikit-learn`. En el anterior ejemplo hemos usado el valor por defecto que usa `scikit-learn` (`alpha=1`). El valor óptimo de `alpha` dependerá de los datos que usemos. Incrementarlo llevará a tener peor ajuste en el entrenamiento pero puede llevar a mejor generalización. Vamos a aprender a usarlo en los ejercicios.

Incrementar `alpha` significa hacer que los pesos o coeficientes estén más restringidos.

# Regularización Lasso

Una alternativa a Ridge para regularizar sería Lasso (Least Absolute Shrinkage and Selection Operator). 

La diferencia principal entre Ridge y Lasso es que Lasso permite que algunas columnas o *features* de nuestro dataset queden anuladas, es decir, las multiplica por cero para que no se utilicen en la regresión lineal. Esto permite hacer selección de *features* permitiendo eliminar algunas de ellos y que nuestro modelo sea más explicable al tener menos dimensiones.

La fórmula de cálculo de esta regularización se muestra a continuación:

$$ f(w, b) = \lambda|\textbf{w}|+\frac{1}{N}\sum_{i=1}^{N}(y_i - (w*x_i+b))^2 $$

donde $ |\textbf{w}| = \sum_{j=1}^{D}|w_{j}| $, es la suma del valor absoluto de los pesos
y $ \lambda $ es un hiperparámetro que controla la importancia de la regularización.

Si $ \lambda = 0 $ nos encontramos en el caso de la regresión lineal. En cambio, si $ \lambda >> 0 $ el algoritmo de aprendizaje intentará que todos los $ w_j $ estén muy cerca de 0 o siendo 0 y el modelo puede acabar siendo muy simple y puede acabar en un subajuste (*underfitting*).


In [12]:
from sklearn.linear_model import Lasso
linreg_lasso = Lasso(alpha=1.0).fit(X_train, y_train)

Miremos ahora cómo se comporta en los datos de entrenamiento:

In [13]:
print(linreg_lasso.score(X_train, y_train))

0.29323768991114607


Los resultados son muy malos, ¿y en el conjunto de datos de prueba?

In [14]:
print(linreg_lasso.score(X_test, y_test))

0.20937503255272294


¿Por qué está pasando esto?, ¿cómo es posible que funcione tan mal?

Si miramos el total de *features* que está seleccionando Lasso lo podemos entender fácilmente:

In [15]:
print(np.sum(linreg_lasso.coef_ != 0))

4


¡Únicamente está utilizando 4 columnas de todas las columnas disponibles! (más de 100)

Como en el caso de `Ridge`, `Lasso` también tiene su hiperparámetro, la $ \lambda $ que podemos toquetear, que, al igualque en el caso de `Ridge`, en `scikit-learn` se llama `alpha`. El valor por defecto vuelve a ser `alpha=1`. Vimos que en ambas regularizacionea cuando incrementábamos el valor de $ \lambda $ (`alpha` en `scikit-learn`) los valores tendían más a 0. En el caso que estamos viendo quizá sea mejor usar un valor entre 0 y 1 porque incrementar aun más `alpha` nos podría dar incluso peores resultados todavía.

Probemos con el valor 0.01:

In [16]:
linreg_lasso001 = Lasso(alpha=0.01).fit(X_train, y_train)

/Users/jordi/anaconda3/envs/pycones19/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 46.90245800318007, tolerance: 3.233182126649076
  positive)


La anterior advertencia nos indica que quizá debemos aumentar el número de iteraciones para que los valores converjan a una tolerancia aceptable. Vamos a hacer caso a los expertos:

In [17]:
linreg_lasso001 = Lasso(alpha=0.01, max_iter=100_000).fit(X_train, y_train)

El error en este caso sobre los datos de entrenamiento es:

In [18]:
print(linreg_lasso001.score(X_train, y_train))

0.8962226511086497


Y sobre los datos de test:

In [19]:
print(linreg_lasso001.score(X_test, y_test))

0.7656571174549982


Y el número de columnas seleccionadas es:

In [20]:
print(np.sum(linreg_lasso001.coef_ != 0))

33


¡Esto ya es otra cosa! El modelo es más complejo porque tiene más dimensiones pero parece mucho más útil que nuestro intento inicial.

# Ejercicios

**Ejercicio 1**. En la regularización Ridge hemos probado únicamente el valor por defecto `alpha=1`. Realiza la misma ejecución con `alpha=0.1` y `alpha=0.01` y razona sobre los resultados, ¿qué valor deberíamos escoger? 

Ejemplo de solución: 

(**NOTA**: ejecuta la celda para cargar el código)

In [ ]:
%load ../../solutions/03_01_ridge.py

**Ejercicio 2**. Después de la ejecución anterior, imagínate que queremos saber qué valor de alpha sería el más adecuado entre todas las posibilidades entre 0.005 y 1 con incrementos de 0.005, para asegurarnos que estamos escogiendo el mejor valor posible.

Para ello, puedes utilizar el método [sklearn.linear_model.RidgeCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html) al que podemos pasar una lista de valores alphas y nos devolverá el alpha con el mejor resultado. Así de sencillo.

Por ejemplo:

In [ ]:
from sklearn.linear_model import RidgeCV

alphas = [1, 0.1, 0.01]
linreg_ridge_cv = RidgeCV(alphas).fit(X_train, y_train)
print(linreg_ridge_cv.alpha_)

¡Ahora te toca a ti!

In [ ]:
%load ../../solutions/03_02_ridgecv.py

**Ejercicio 3**. Como hemos comentado, la regularización Lasso permite realizar una selección de las columnas. En concreto, Lasso retiene las *features* que considera más adecuadas para ajustar el modelo creado a los datos de entrenamiento, según el parámetro alpha especificado. Por lo tanto, cambiando el parámetro, obtendremos un conjunto u otro de columnas. Y además, si cambiamos los datos de entramiento, podríamos con el mismo parámetro alpha incluso tener una selección diferente de columnas.

En este sentido, comprueba ahora qué resultado tenemos mediante una regularización Lasso para los valores 0.005 y 0.5 y qué número de columnas selecciona para cada caso, ¿qué conclusión podemos sacar de los valores obtenidos? Utiliza en ambos casos 100.000 iteraciones máximas.

In [ ]:
%load ../../solutions/03_03_lasso.py

# Desarrollo matemático

Los dos tipos de regularización más ampliamente usados se llaman regularización L1 y regularización L2. La idea es bastante simple, para crear un modelo regularizado modificamos la función objetivo añadiendo un término de penalización cuyo valor es más alto cuando el modelo es más complejo.

Por simplicidad, vamos a ver esto de la regularización usando la regresion lineal. El mismo principio puede ser aplicado a una amplia variedad de modelos.

Vamos a recordar la función objetivo a minimizar de la regresión lineal:

$$ f(w, b) = \frac{1}{N}\sum_{i=1}^{N}(y_i - (w*x_i+b))^2 $$

A la anterior función le metemos un término que penaliza la función de pérdida.

Una función objetivo regularizada usando el tipo L1 sería de la siguiente forma:

$$ f(w, b) = \lambda|\textbf{w}|+\frac{1}{N}\sum_{i=1}^{N}(y_i - (w*x_i+b))^2 $$

donde $ |\textbf{w}| = \sum_{j=1}^{D}|w_{j}| $, es la suma del valor absoluto de los pesos
y $ \lambda $ es un hiperparámetro que controla la importancia de la regularización.

Si $ \lambda = 0 $ nos encontramos en el caso de la regresión lineal. En cambio, si $ \lambda >> 0 $ el algoritmo de aprendizaje intentará que todos los $ w_j $ estén muy cerca de 0 o siendo 0 y el modelo puede acabar siendo muy simple y puede acabar en un subajuste (*underfitting*). Tu trabajo va a ser ajustar correctamente este hiperparámetro.

De la misma forma, una función objetivo regularizada usando el tipo L2 sería de la siguiente forma:

$$ f(w, b) = \lambda||\textbf{w}||^2 +\frac{1}{N}\sum_{i=1}^{N}(y_i - (w*x_i+b))^2 $$

donde $ ||\textbf{w}|| = \sum_{j=1}^{D}(w_{j})^2 $, es la suma del valor al cuadrado de los pesos
y $ \lambda $ sigue siendo un hiperparámetro que controla la importancia de la regularización.

Al igual que antes, si $ \lambda = 0 $ nos encontramos en el caso de la regresión lineal. En cambio, si $ \lambda >> 0 $ el algoritmo de aprendizaje intentará que todos los $ w_j $ estén muy cerca de 0 (pero no siendo cero).

La diferencia básica entre la regularización L1 y la regularización L2 es que en el caso de la primera varios pesos acaban siendo cero ayudando a mostrar qué *features* van a ser importantes en el ajuste. Nos permite hacer *feature selection* lo que permite que nuestro modelo sea más explicable por la simplificación. El L2 generaliza más y suele dar mejores resultados. En la literatura se puede encontrar que la regularización L1 se le llama también **Lasso** y la regularización L2 se le llama también **Ridge**.

Se pueden combinar las regularizaciones L1 y L2 (por ejemplo, *elastic net*). Estas regularizaciones se usan ampliamente en modelos lineales pero también en redes neuronales y otros tipos de modelos.

L2 también nos permite resolverlo usando Gradiente Descendente ya que es diferenciable. En cambio, Lasso (y ElasticNet) no usan gradiente descendiente para minimizar su función de coste. Esto es debido a que $ |\textbf{w}| $ no es diferenciable y no podemos usar Gradiente Descendente. En este caso usan un algoritmo de optimización que se llama [*Coordinate descent*](https://en.wikipedia.org/wiki/Coordinate_descent).

Existen otros tipos de regularizaciones que no vamos a ver como *dropout*, *batchnormalization*, *data augmentation* o *early stopping*.

# Referencias

* http://scott.fortmann-roe.com/docs/BiasVariance.html
* https://trainingdatascience.com/training/401-linear-regression/
* https://www.thelearningmachine.ai/lle